COMP 215 - LAB 3 Classes (NEO)
----------------
#### Name: Nika Ghassemi
#### Date: Sun Jan 18 2026

This lab exercise introduces `class` as a means of organizing related data and functions.

**Building on new concepts from lab 2**:
  * a `record` is a related collection of data, with fields for each data value
  * an `API` is an "Application Programmers Interface" defining how a programmer interacts with a system.
  * *f-string* simplifies string formatting operations

**New Python Concepts**:
  * the `class` keyword allows you define a new data `type`, with a set of operations on that data.
  * a `dataclass` simplifies class definition for classes that primarily encapsulate a data structure.

As usual, the first code cell simply imports all the modules we'll be using...

In [73]:
import datetime, json, requests
from pprint import pprint    # Pretty Print - built-in python function to nicely format data structures

We'll continue working with [Near Earth Object](https://cneos.jpl.nasa.gov/) data
> using NASA's API:  [https://api.nasa.gov/](https://api.nasa.gov/#NeoWS)

Here's a brief review from Lab 2 on how to use it...

### Review: making a query

Here's a query that gets the record for a single NEO that recently passed by.

In [74]:
API_KEY = 'DEMO_KEY'  # substitute your API key here

def get_neos(start_date):
    """ Return a list of NEO for the week starting at start_date """
    url = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&api_key={API_KEY}'
    # Fetch last week's NEO feed
    response = requests.request("GET", url, headers={}, data={})
    data = json.loads(response.text)
    return [neo for dated_records in data['near_earth_objects'].values() for neo in dated_records ]

def get_neo(id):
    """ Return a NEO record for the given id """
    url = f'https://api.nasa.gov/neo/rest/v1/neo/{id}?api_key={API_KEY}'
    response = requests.request("GET", url, headers={}, data={})
    return json.loads(response.text)

# Sample usage:  get the list of NEOs for a given week, then lookup the latest NEO record in that list.
week_start = '2023-01-15'
neos = get_neos(week_start)
print(f'{len(neos)} Near Earth Objects found for week of {week_start}')
assert len(neos) > 0, f'Oh oh!  No NEOs found for {week_start}'
neo = get_neo(neos[-1]['id'])  # get the very latest NEO
pprint(neo)

113 Near Earth Objects found for week of 2023-01-15
{'error': {'code': 'OVER_RATE_LIMIT',
           'message': 'You have exceeded your rate limit. Try again later or '
                      'contact us at https://api.nasa.gov:443/contact/ for '
                      'assistance'}}


## Exercise 1: Define a CloseApproach class

Each NEO record comes with a list of `close_approach_data`, where each record in this list represents a single “close approach” to another orbiting body.
* Develop a class named `CloseApproach` to represent a *single* close approach record.
* State variables are
    * orbiting body (`str`)
    * approach date (`datetime` object!)
    * miss distance (`float` in km, document it!)
    * relative velocity (`float` in km/hr, ditto)

* Operations must include:
    * `__init__(self, ...)` method to initialize a new object with specific data values
    * `__str__(self)` method to return a nicely formatted string representation of the object.

Write a little code to test your new class.

In [63]:
# Ex. 1 your code here
class CloseApproach:
  def __init__(self, orbiting_body, approach_date, miss_distance, relative_velocity):
    self.orbiting_body = orbiting_body
    self.approach_date = approach_date
    self.miss_distance = miss_distance
    self.relative_velocity = relative_velocity

  def __str__(self):
    return f"orbiting_Body: {self.orbiting_body}, approach_date: {self.approach_date}, miss_distance: {self.miss_distance}, relative_velocity: {self.relative_velocity}"


# Test code for CloseApproach class
approach = CloseApproach(
        orbiting_body="Earth",
        approach_date="2026-01-18",
        miss_distance=384400,
        relative_velocity=54000
    )

print(approach)

orbiting_Body: Earth, approach_date: 2026-01-18, miss_distance: 384400, relative_velocity: 54000


## Exercise 2: Factory function: get_close_approach

We want to be able to construct CloseApproach objects easily from a data record returned from the NEO API.  

Write an "object factory" function...   

    def get_close_approach(record):
        ...

This function provides an easy way to create a `CloseApproach` instance.  It takes a dictionary for a single `close_approach_data` record, constructs and returns a `CloseApproach` object representing that same record.
This kind of function is called a “Factory” because it handles the details of constructing an object from raw materials.

Remember to convert each element from the data dictionary to the correct type (e.g., parse the date/time string into a `datetime` object).
Add little code to test your new factory function.

In [64]:
# Ex. 2 your code here
def get_close_approach(record):
    orbiting_body = record['orbiting_body']
    approach_date = record['close_approach_date']
    approach_time = record['close_approach_time']
    miss_distance = record['miss_distance']['kilometers']
    relative_velocity = record['relative_velocity']['kilometers_per_hour']

    return CloseApproach(orbiting_body, approach_date, miss_distance, relative_velocity)

    sample_record = {
        "orbiting_body": "Earth",
        "close_approach_date": "2026-01-18",
        "miss_distance_km": "384400",
        "relative_velocity_kph": "54000"
    }
    print(get_close_approach(sample_record))

## Exercise 3:  Define an Asteroid class

Define a simple Asteroid class with some basic state variables representing a single NEO.  Your Asteroid class should define at least 4 "state variables:”
* id  (`int`)
* name (`str`)
* estimated_diameter (`float` in m)
* is_potentially_hazardous (`bool`)
* close_approaches (`list` of CloseApproach objects, default to empty list)

Operations must include:
* `__init__(self, ...)` method to initialize a new object with specific data values
* `__str__(self)` method to return a nicely formatted string representation of the object.

Write a little code to test you new class (just leave close_approaches as an empty list for now).

In [71]:
# Ex. 3 your code here
class Asteroid:
  def __init__(self, id, name, estimated_diameter, is_potentially_hazardous, close_approaches=[]):
    self.id = id
    self.name = name
    self.estimated_diameter = estimated_diameter
    self.is_potentially_hazardous = is_potentially_hazardous
    self.close_approaches = close_approaches

  def __str__(self):
    return f"id: {self.id}, name: {self.name}, estimated_diameter: {self.estimated_diameter} m, is_potentially_hazardous: {self.is_potentially_hazardous}"


# Test code for Asteroid class
test_asteroid = Asteroid(
    id="3542519",
    name="(2010 PK6)",
    estimated_diameter="120.5",
    is_potentially_hazardous=True
)

print(test_asteroid)

id: 3542519, name: (2010 PK6), estimated_diameter: 120.5 m, is_potentially_hazardous: True


## Exercise 4: Asteroid factory

Write a function that returns an Asteroid object just from the id for a single NEO.

    def asteroid_from_neo(neo_id):
        ...

This factory function takes the `id` for a single NEO, fetches the NEO record from API, constructs and returns an Asteroid object representing that NEO.  *Hint*: I provided the code fetch a NEO from its `id` above.

Every `Asteroid` should have a list of “close approaches”.
*Hint*: use the `get_close_approach` factory you defined above to construct the required list of CloseApproach objects.

Now add a new method to `Asteroid` class to return the `CloseApproach` object from the asteroid representing its nearest to **Earth**:

    def nearest_miss(self):
        ...

Extend your test code to demonstrate these new features.

In [70]:
# Ex. 4 your test code for nearest_miss here
def asteroid_from_neo(neo_id):
    neo = get_neo(neo_id)
    close_approaches = [get_close_approach(approach) for approach in neo['close_approach_data']]
    return Asteroid(
        id=neo['id'],
        name=neo['name'],
        estimated_diameter=neo['estimated_diameter']['kilometers']['estimated_diameter_max'],
        is_potentially_hazardous=neo['is_potentially_hazardous_asteroid'],
        close_approaches=close_approaches
    )

## Challenge - Take your skills to the next level...
### Exercise 5: develop a useful analysis / data product

 With these data structures in place, we can now start answering all kinds of interesting questions about a single Asteroid or a set of Asteroids.  
Here’s a couple ideas to try:

* write a **function** named `most_dangerous_approach`, that takes a date range and returns a single “potentially hazardous” Asteroid object that makes the closest approach to Earth within that range.  Your algorithm will ultimately need to:
    * grab the list of NEO’s for the given date range;
    * use a list comprehension to build the list of Asteroid objects for the NEO’s returned
    * use a list comprehension to filter  potentially hazardous Asteroids only;
    * use a list comprehension to map each Asteroid to its  nearest_miss
    * apply Python’s min function to identify the Asteroid with the nearest_miss

You may want to decompose some of these steps into smaller functions.
* add a method to the Asteroid class, `estimated_mass`, that computes an estimate of the Asteroid’s mass based on its diameter.  This is a model – state your assumptions.
* add a method to the `CloseApproach` class, `impact_force`,  that estimates the force of impact if the Asteroid hit the orbiting object.  Again, this is a model, state your assumptions.

In [ ]:
# Ex. 5 (challenge) your code here